In [ ]:
import numpy as np
import cv2
from skimage import exposure
from skimage.feature import canny
from skimage.transform import hough_circle, hough_circle_peaks
from skimage.draw import circle_perimeter
import matplotlib.pyplot as plt
import spectral as sp
import os
from skimage.measure import label, regionprops

In [ ]:
# Step 1: Load hyperspectral data
def load_hyperspectral_data(bil_path):
    
    hsi = sp.open_image(bil_path)
    hsi_data = hsi.load()
    
    return hsi_data


# Step 2: Flat-field correction
def flat_field_correction(raw_cube, dark_ref, white_ref):
    
    corrected_cube = (raw_cube - dark_ref) / (white_ref - dark_ref)
    
    return corrected_cube


# Step 3: Calculate Normalized Band Difference (NBD)
def calculate_nbd(reflectance_cube, band1_idx, band2_idx):
    
    r_band1 = reflectance_cube[:, :, band1_idx]
    r_band2 = reflectance_cube[:, :, band2_idx]
    
    # Avoid division by zero
    epsilon = 1e-10
    denominator = r_band2 + r_band1 + epsilon
    
    nbd_image = (r_band2 - r_band1) / denominator
    return nbd_image


# Step 4: Enhance Image Contrast
def enhance_contrast(image):
    
    return exposure.equalize_hist(image)


# Step 5: Segment using thresholding
def segment_image(image):
    
    _, binary_mask = cv2.threshold((image * 255).astype('uint8'), 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
    
    return binary_mask


# Step 6: Morphological operations
def clean_segmentation(binary_mask):
    
    kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (3, 3))
    
    cleaned_mask = cv2.erode(binary_mask, kernel, iterations=2)
    cleaned_mask = cv2.dilate(cleaned_mask, kernel, iterations=2)
    
    return cleaned_mask



def detect_circles(cleaned_mask, radius_range):
    
    # Hough Circle Transform
    hough_radii = np.arange(radius_range[0], radius_range[1], 1)
    hough_res = hough_circle(cleaned_mask, hough_radii)

    # Identify the most prominent circles
    accums, cx, cy, radii = hough_circle_peaks(hough_res, hough_radii, total_num_peaks=600)

    # Create a mask to keep only the circular regions
    circle_mask = np.zeros_like(cleaned_mask, dtype=bool)
    
    for center_y, center_x, radius in zip(cy, cx, radii):
        
        rr, cc = circle_perimeter(center_y, center_x, radius, shape=circle_mask.shape)
        circle_mask[rr, cc] = True

    # Fill the circles to create a binary mask
    filled_circle_mask = np.zeros_like(cleaned_mask, dtype=np.uint8)  # Convert to uint8
    
    for center_y, center_x, radius in zip(cy, cx, radii):
        
        rr, cc = circle_perimeter(center_y, center_x, radius, shape=filled_circle_mask.shape)
        filled_circle_mask[rr, cc] = 255  # Set perimeter pixels to 255
        
        cv2.circle(filled_circle_mask, (center_x, center_y), radius, 255, thickness=-1)  # Fill circle with 255

    return filled_circle_mask



def segment_and_save_blueberries(filled_circle_mask, reflectance_cube, output_dir, box_size=125):
    # Label connected components in the circular mask
    labeled_mask = label(filled_circle_mask)
    regions = regionprops(labeled_mask)

    # Ensure the output directory exists
    os.makedirs(output_dir, exist_ok=True)

    # Loop through each labeled region
    half_box = box_size // 2  # Half of the desired box size
    for idx, region in enumerate(regions):
        # Get the centroid of the region
        center_y, center_x = region.centroid

        # Calculate fixed bounding box
        min_row = max(int(center_y) - half_box, 0)
        max_row = min(int(center_y) + half_box, reflectance_cube.shape[0])
        min_col = max(int(center_x) - half_box, 0)
        max_col = min(int(center_x) + half_box, reflectance_cube.shape[1])

        # Create a mask for this specific region
        individual_mask = (labeled_mask[min_row:max_row, min_col:max_col] == region.label)

        # Extract the corresponding hyperspectral cube region
        blueberry_cube = reflectance_cube[min_row:max_row, min_col:max_col, :]

        # Apply the mask to the hyperspectral data
        blueberry_cube *= individual_mask[:, :, np.newaxis]

        # Check if the extracted cube matches the required size (pad if necessary)
        if blueberry_cube.shape[0] != box_size or blueberry_cube.shape[1] != box_size:
            blueberry_cube = np.pad(
                blueberry_cube,
                (
                    (0, max(0, box_size - blueberry_cube.shape[0])),  # Pad rows
                    (0, max(0, box_size - blueberry_cube.shape[1])),  # Pad cols
                    (0, 0),  # No padding for spectral dimension
                ),
                mode='constant',
                constant_values=0
            )

        # Save the extracted cube as an .npy file
        output_path = os.path.join(output_dir, f"blueberry_{idx+1}.npy")
        np.save(output_path, blueberry_cube)

        # Optionally visualize the region (for debugging/verification)
        plt.imshow(np.mean(blueberry_cube, axis=2), cmap='gray')  # Mean over spectral bands
        plt.title(f"Blueberry {idx+1} (Masked, 125x125)")
        plt.show()

    print(f"Saved {len(regions)} blueberries to {output_dir}")

In [ ]:
# Main Function
def process_hyperspectral_cube(file_path, white_ref_path, dark_ref, band1_idx, band2_idx, radius_range):
    
    # Load raw data and white reference
    raw_cube = load_hyperspectral_data(file_path)
    white_ref = load_hyperspectral_data(white_ref_path)
    
    # Visualize raw cube
    plt.imshow(raw_cube[:, :, band1_idx], cmap='gray')
    plt.title("Raw Cube (Band {})".format(band1_idx))
    plt.show()
    
    # Flat-field correction
    reflectance_cube = flat_field_correction(raw_cube, dark_ref, white_ref)
    plt.imshow(reflectance_cube[:, :, band1_idx], cmap='gray')
    plt.title("Flat-Field Corrected Cube (Band {})".format(band1_idx))
    plt.show()
    
    # Calculate NBD
    nbd_image = calculate_nbd(reflectance_cube, band1_idx, band2_idx)
    plt.imshow(nbd_image, cmap='gray')
    plt.title("Normalized Band Difference (NBD)")
    plt.colorbar()
    plt.show()
    
    # Enhance contrast
    enhanced_image = enhance_contrast(nbd_image)
    plt.imshow(enhanced_image, cmap='gray')
    plt.title("Enhanced Contrast Image")
    plt.colorbar()
    plt.show()
    
    # Segment
    segmented_mask = segment_image(enhanced_image)
    plt.imshow(segmented_mask, cmap='gray')
    plt.title("Segmented Binary Mask")
    plt.show()
    
    # Clean segmentation
    cleaned_mask = clean_segmentation(segmented_mask)
    plt.imshow(cleaned_mask, cmap='gray')
    plt.title("Cleaned Segmentation Mask")
    plt.show()
    
    # Detect circles
    final_mask = detect_circles(cleaned_mask, radius_range)
    plt.imshow(final_mask, cmap='gray')
    plt.title("Detected Circles")
    plt.show()
    
    output_dir = "Good_Blueberry_169_210" #must change according to dataset 
    #segment_and_save_blueberries(final_mask, reflectance_cube, output_dir)
    segment_and_save_blueberries(final_mask, reflectance_cube, output_dir, box_size=125)

    return final_mask

In [ ]:
# Dataset 
#must change according to dataset files
dataset_path = '<path_to_file>/Original'

bil_file_path = os.path.join(dataset_path, "Good Blueberry 169-210.bil.hdr") #must change according to dataset 
white_ref_path = os.path.join(dataset_path, "WhiteReference_1600.bil.hdr") #must change according to dataset 

near_zero = 1e-6
dark_ref = np.full((1600, 1600, 462), near_zero) #must change according to dataset 

band1_idx = 216   #680nm
band2_idx = 380   #900nm
radius_range = (40, 85)

final_mask = process_hyperspectral_cube(bil_file_path, white_ref_path, dark_ref, band1_idx, band2_idx, radius_range)